# **Custos Médicos - Modelos de Regressão**

A ideia neste problema é estimar o custo do seguro no plano de saúde para uma pessoa com base em dados adicionais como idade, se é fumante ou não, se tem dependentes, etc.

Isto quer dizer que temos considerar que nuestra variável dependiente será o **_Custo_** e as caraterísticas restantes serão consideradas como as variáveis independientes, em outras palavras o **Custo** depende (umas vezes mais e outras menos) de todas as outras variáveis

O Primeiro a fazer é estabelecer a influencia de todas essas variáveis (caraterísticas) no valor final dos Custos Médicos, de forma individual.

**_É importante lembrar que temos que ter a capacidade de predecir um valor de custo para uma nova pessoa (cliente) com base as suas proprias caraterísticas_.**

Iniciemos esta análise selecionando os dados que vamos a utilizar.

# Importação dos dados:
Os dados podem ser obtido do site do Kaggle.com : https://www.kaggle.com/datasets/mirichoi0218/insurance

Podemos fazer um download e depois abrir ou também importar diretamente usando um link onde esta o arquivo. Esses dados serão guardados na variável **dados** do tipo _dataframe_.

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/profjuanito/Dataset/main/insurance.csv'
dados = pd.read_csv(url)
dados.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# Informações dos dados e Mise en place:

Obter informações gerais dos dados como número de variáveis, tipo, etc. Os comandos podem ser visualizados e entendidos melhor nos anteriores scripts (Custos_Medicos.ipynb, etc)

Lembrar o que foi feito nos modelos anteriores:

* Transformar a variável **Fumante** em 0 (não) ou 1 (sim)
* Colocar a variável resposta na escala para cada mil dolares.

Nesta análise também vamos considerar só as variáveis:
* var. dependiente: **Custos**
* var. independientes: indice de massa corporal (**IMC**) e ser fumante ou não.

In [ ]:
# mise en place
dados['smoker'] = dados['smoker'].map({'yes': 1, 'no': 0})
dados['charges'] = dados['charges']/1000 
dados['sex'] = dados['sex'].map({'male': 1, 'female': 0})

dados = dados[['age', 'bmi', 'smoker','sex', 'charges']]
dados.head()

,age,bmi,smoker,sex,charges
0,19,27.900,1,0,16.884924
1,18,33.770,0,1,1.725552
2,28,33.000,0,1,4.449462
3,33,22.705,0,1,21.984471
4,32,28.880,0,1,3.866855


---
---

## **Comparação dos Modelos**

## Técnica de Validação: _Holdout Repetido_

Por ser aleatoria a seleção dos conjuntos, é melhor fazer varias vezes este procedimento e assim obter uma estatística para calcular o valor mais estável para o EQM.

Como será feita uma comparação entre os modelo: **Arvore de decisão** (com diferentes valores de profundidades), de **K-NN** (com diferentes valores de k) e de **Regressão Linear** é preferível criar uma função a qual vai chamar cada modelo com seus respectivos parâmetros. 

In [ ]:
def HoldoutRepetido(modelo, replicas = 100, proporcao = 0.25): 

    eqms_treino = []
    eqms_teste = []

    for i in range(replicas):
        Xtreino , Xteste, ytreino, yteste  = train_test_split(X, y, test_size = proporcao)
        
        #Treinando o Modelo
        reg = modelo
        reg.fit(Xtreino, ytreino)

        #Verificando o eqm no teste
        preditos_treino = reg.predict(Xtreino) 
        eqm_treino = mean_squared_error(ytreino, preditos_treino)

        preditos_teste = reg.predict(Xteste)
        eqm_teste = mean_squared_error(yteste, preditos_teste)

        #Guardando o resultado
        eqms_treino.append(eqm_treino)
        eqms_teste.append(eqm_teste)
    
    res_treino = np.array(eqms_treino).mean().round(2)
    res_teste = np.array(eqms_teste).mean().round(2)

    print('EQM médio de TREINO: {}'.format(res_treino))
    print('EQM médio de TESTE: {}\n'.format(res_teste))

    return res_treino, res_teste 

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split 
import numpy as np

X = dados[['age','bmi','smoker']]
y = dados['charges'] 

modelos = [LinearRegression(), 
           KNeighborsRegressor(n_neighbors = 5), 
           KNeighborsRegressor(n_neighbors = 15),
           KNeighborsRegressor(n_neighbors = 25),
           KNeighborsRegressor(n_neighbors = 35),
           KNeighborsRegressor(n_neighbors = 45),
           DecisionTreeRegressor(max_depth = 1),
           DecisionTreeRegressor(max_depth = 2),
           DecisionTreeRegressor(max_depth = 3),
           DecisionTreeRegressor(max_depth = 4),
           DecisionTreeRegressor(max_depth = 5)]

nomes = ['Regressão Linear M',
         'KNN - K = 5', 
         'KNN - K = 15', 
         'KNN - K = 25', 
         'KNN - K = 35', 
         'KNN - K = 45',
         'Árvore - Prof = 1',
         'Árvore - Prof = 2',
         'Árvore - Prof = 3',
         'Árvore - Prof = 4',
         'Árvore - Prof = 5']

treinos = []
testes = []

for modelo in modelos:
    resultados = HoldoutRepetido(modelo)
    treinos.append(resultados[0])
    testes.append(resultados[1])

aux = {'EQM Treino': treinos, 'EQM Teste': testes}
resultados = pd.DataFrame(aux, index = nomes)
resultados.sort_values(by = 'EQM Teste')

EQM médio de TREINO: 36.93
EQM médio de TESTE: 37.42

EQM médio de TREINO: 72.13
EQM médio de TESTE: 116.79

EQM médio de TREINO: 108.53
EQM médio de TESTE: 125.4

EQM médio de TREINO: 118.05
EQM médio de TESTE: 128.44

EQM médio de TREINO: 121.17
EQM médio de TESTE: 128.77

EQM médio de TREINO: 123.01
EQM médio de TESTE: 128.36

EQM médio de TREINO: 55.86
EQM médio de TESTE: 55.46

EQM médio de TREINO: 25.2
EQM médio de TESTE: 26.22

EQM médio de TREINO: 20.87
EQM médio de TESTE: 22.3

EQM médio de TREINO: 19.13
EQM médio de TESTE: 22.4

EQM médio de TREINO: 17.78
EQM médio de TESTE: 23.56



,EQM Treino,EQM Teste
Árvore - Prof = 3,20.87,22.30
Árvore - Prof = 4,19.13,22.40
Árvore - Prof = 5,17.78,23.56
Árvore - Prof = 2,25.20,26.22
Regressão Linear M,36.93,37.42
Árvore - Prof = 1,55.86,55.46
KNN - K = 5,72.13,116.79
KNN - K = 15,108.53,125.40
KNN - K = 45,123.01,128.36
KNN - K = 25,118.05,128.44


---

## Conclusão

A inclusão da **Idade**  como variável independente diminui o valor do **EQM**, caindo da casa dos 30 para 20 na maioria dos casos do **Árvores de Decisão**, mostrando que a variável **Idade** tem que ser considerada. Para a **Regressão Linear**  tambén tivemos uma melhora , caindo de **EQM** de 50 para 37. Infelizmente a inclusão desta novva variável (**Idade**) fez que a fosse perdida a o valor do **EQM**  de 30 para 70 e  

A realização dos teste de validação mostra que o nosso erro médio quadrático (**eqm**) é consistente, pudendo dizer que nos resultados obtidos, a pesar da ser executado muitas vezes com diferentes conjuntos treino e teste, mostram uma dispersão pequena.

Ainda poderíamos tentar melhorar trocando a **Regressão Linear Multipla** por uma regressão com uma função **Não Linear**. Isto será estudado posteriormente.

---